### MQTT Blaster ###

In [8]:
#!/usr/bin/env python3
import paho.mqtt.client as mqtt
import time
from timeit import default_timer as timer
from datetime import timedelta
import multiprocessing
import numpy as np
import getopt, sys
import subprocess, os

# This is the Publisher
major_number=0
minor_number=1
version_info="mqtt-blaster: V"+str(major_number)+"."+str(minor_number)

# mqtt configurations
SERVER_IP = "192.168.29.149"
#SERVER_IP = "ec2-3-81-9-181.compute-1.amazonaws.com"
PORT = 1883
KEEP_ALIVE = 60
QOS0 = 0
QOS1 = 1
QOS2 = 2
RETAIN = False

QOS = QOS2

# number of topics/publishers
max_publishers = 3
max_topics = 1

topic_root = "topic/"
topic_latency = topic_root+"latency"
topic_summary = topic_root+"summary"

# approximate message rate. sleep time is 1 sec/rate (40000 is max rate)
# for loop constant is 35171223
msg_count = 250000
max_msg_rate = 40000
for_constant = 23101223
sleep_interval = int(for_constant/(max_msg_rate/max_publishers))

not_end = 1

def usage():
    print(version_info)
    print("\nUsage: $ mqtt-blaster [options]... -s [ipaddress/hostname]\n")
    print("Options:\n--------")
    print("-q, qos: [0, 1, 2], default: 0")
    print("-r, rate of message [1-6], default: 1 ")
    print("-c, message count [1-1000000], default: 1 ")
    print("-p, number of publishers to be used [1-4], default: 1")
    print("-t, number of topics [1-4], default: 1") 
    print("-v, print version info") 
    print("-h, print this help")
    print("-a, try remote/local performace script installation,[0,1] default: 0 (do not try)")
    print("    this option requires, keyless authentication enabled between client and server")
    print("    Note: enabling keyless authentication is an onetime activity 'and' is applicable to")
    print("          locally deployed servers. Public deployments may not allow ssh logins or ")
    print("          messuring server KPIs")
    print("Dependencies\n------------")
    print("Following tools are required on server and client(publisher) side")
    print("to measure various system parameters during experiment:")
    print("python 3.x, top, cpustat, a, b, c")
    pass

def on_publish_publisher(client, userdata, msg):
    #print("on_publish")
    pass

# this is the publisher thread
def publisher_process(publisher_id):
    print("Publisher [" + str(publisher_id) + "] started...\r")
    client = mqtt.Client(clean_session=True)
    #client.max_inflight_messages_set(0)
    client.on_publish = on_publish_publisher
    client.connect(SERVER_IP, PORT, KEEP_ALIVE)

    # time to settle down
    time.sleep(2)

    # setup message
    #message_prefix = "PUB" + str(publisher_id) + "." + topic_root
    message_prefix = "PUB" + str(publisher_id) + "." + topic_root + str(publisher_id) + ": m#"
    #padding = ".Ac#4d%^NHTyAUDkajYRASIKDag98124326810: m#"
    padding = ".Ac#4d%^NHTyAUDkajYRASIKDag98124326810: m#"
    message_prefix += padding
    
    #client.loop_start()
    # exp start time
    start = timer()
   
    # each publisher publishes 'msg_count' number of messages per topic
    #changing to topic per publishe
    topic = topic_root + str(publisher_id)
    ###message_no = 0
    
    ### while(message_no < msg_count):
        ###latency_calc_sub.loop_start()
        ###message = message_prefix + str(message_no) + "\r"
        ###client.publish(topic, message, qos=QOS, retain=RETAIN)
        ###message_no += 1
        
    client.loop_start()
    
    for message_no in range(msg_count):
        #for topic_id in range(max_topics):
            # prepare topic and message
            #topic = topic_root + str(topic_id)
            #message = message_prefix + str(topic_id) + "." + str(message_no) + ":" +str(timer()) + "\r"
        #message = message_prefix + "." + str(message_no) + ":" +str(timer()) + "\r"
        message = message_prefix + str(message_no) + "\r"
            
            # publish the message
        client.publish(topic, message, qos=QOS, retain=RETAIN)
        # time to sleep/rate control. (below)
        # number to get 100% cpu utilization on server.
        # mqtt: qos1: , qos2: 1000
        #for i in range (800):
        #    pass
        time.sleep(0.0002)

    # exp end time
    end = timer()
    elapsed_time = end -start

    # now publish the numbers: each publisher publishes total number messages sent on all topics
    # currently publishing summary on topic_summary channel. 
    time.sleep(2)
    message = "\nSummary\n-------\nPublisher: " + str(publisher_id)+"\n" \
    + "Max Topics: " + str(max_topics) \
    + "\nMax Topics: " + str(max_publishers) \
    + "\nMessages per Topic: " + str(msg_count) \
    + "\nTotal Time: " + str(elapsed_time) \
    + "\nActual rate: " + str(max_topics*msg_count/(elapsed_time)) + " messages/second\n"
    
    #client.publish(topic_summary, message,  qos=QOS, retain=RETAIN)
    print(message)
    
    #client.loop_stop()
    # disconnect publishing thread
    client.disconnect();
    
cmd_fetch_srv_stats = ['ssh', '-t', 'root@192.168.29.149', '/usr/share/mqtt/stats-3.sh']

def execute_shell_cmd(cmd):
    process = subprocess.Popen(cmd, 
                           stdout=subprocess.PIPE,
                           universal_newlines=True)
    buf = ""
    while True:
        output = process.stdout.readline()
        buf += output.strip()
        #msg += "\n"+buf
        #print(msg)
        # Do something else
        return_code = process.poll()
        if return_code is not None:
            #print('buf', 'Success:', return_code)
            # Process has finished, read rest of the output 
            for output in process.stdout.readlines():
                buf += output.strip()
                #print(buf)
                #msg += "\n"+buf
            break
    return buf

file="/home/biswa/mqtt/stats0.csv"
header='\"Reading#,Overall Cpu(%),Mqtt cpu usage(%),Disk Read(kb/s),Disk Write(kb/s),Swap Usage(Bytes),Memory(%),NW Rx(Mbps),NW Tx(Mbps), Latency(ms)\"'
cmd_echo_header = 'echo '+header+' > '+file

i = 0
# on_message call back for latency calculations
def on_message(client, userdata, msg):
    global i
    received_time = timer()
    # check for end of experiment message
    if "end" in msg.payload.decode():
        print("Disconnecting latency calculator !")
        client.publish(topic_summary, "Disconnecting latency calculator !\r",  qos=QOS, retain=RETAIN)
        client.disconnect()
    else:
        latency = received_time - float(msg.payload.decode())
        ret = execute_shell_cmd(cmd_fetch_srv_stats)
        msg = str(i)+","+ret+","+str(latency*1000)
        print(msg)
        cmd_echo_message = 'echo '+msg+' >> '+file
        os.system(cmd_echo_message)
        client.publish(topic_summary, msg+"\r",  qos=QOS, retain=RETAIN) 
        i += 1

# stops latency calculator loop.
def on_disconnect(client, userdata,rc=0):
    client.loop_stop()

# latency calculator thread, sends a message with current timestamp to sample latency.
def latency_calculator(num):
    print("Latency calculator started ...\r")
    os.system(cmd_echo_header)
    latency_calc_sub = mqtt.Client()
    latency_calc_sub.on_message = on_message
    latency_calc_sub.on_disconnect = on_disconnect
    latency_calc_sub.connect(SERVER_IP, PORT, KEEP_ALIVE)
    latency_calc_sub.subscribe(topic_latency, QOS)

    print("Waiting for Publishers to start...\r")
    while(not_end):
        latency_calc_sub.loop_start()
        latency_calc_sub.publish(topic_latency, str(timer())+"\r",  qos=QOS, retain=RETAIN)
        time.sleep(2)

if __name__ == "__main__":
    # parse commandline arguments first.
    #if (len(sys.argv)!= 0 ):
        #usage()
        #sys.exit(2)
        # publisher mqtt client object, send 'end' message to latency client, house keeping
        
    publisher_main = mqtt.Client()
    publisher_main.connect(SERVER_IP, PORT, KEEP_ALIVE)
    # send a preamble
    publisher_main.publish(topic_summary, "start\r",  qos=QOS, retain=RETAIN)
    
    # add a latency sampler process
    print("Registering latency Calculator..\r")
    pl = multiprocessing.Process(target=latency_calculator, args=(0,))
    pl.start()

    time.sleep(6)
    
    # publishers
    proc_list = []  
    for n in range(max_publishers):
        # creating publisher process
        print("Registering Publisher[" + str(n) + "]..\r")
        p = multiprocessing.Process(target=publisher_process, args=(n,))
        proc_list.append(p)
        
    exp_start = timer()
    # schedule all procs
    for proc in proc_list:
        proc.start()

    # joins procs
    for proc in proc_list:
        proc.join()
        
    exp_end = timer()
    
    time.sleep(10)
    # notify triggerer to stop processing
    not_end = 0
        
    # send a message on latency channel to stop processing
    print("Wrapping up latency calculator..\r")
    publisher_main.publish(topic_latency, "end\r",  qos=QOS, retain=RETAIN)
    publisher_main.disconnect()
    
    print("PL\r", pl.is_alive())
    if pl.is_alive():
        pl.terminate()
    
    print("Total number of messages: " + str(msg_count * max_topics * max_publishers) +"\r" \
    + "\nTotal Time: " + str(exp_end - exp_start))
    print("Done !")


Registering latency Calculator..
Latency calculator started ...
Waiting for Publishers to start...
0,0,0.0,23.43,62.44,0,15.8461,0,0,1.5506560011999682
1,2.9703,0.0,23.43,62.43,0,15.8444,0,0,6.864874998427695
2,2.94118,0.0,23.43,62.43,0,15.8516,0,0,3.2206539981416427
Registering Publisher[0]..
Registering Publisher[1]..
Registering Publisher[2]..
Publisher [0] started...
Publisher [1] started...
Publisher [2] started...
3,3.92157,5.6,23.43,62.53,0,15.836,0,0,4.922223000903614
4,97.0874,375.0,23.43,62.52,0,15.4863,8.38387,13.9559,3.982096997788176
5,97.0297,375.0,23.42,62.52,0,15.573,8.24471,12.5518,2.6711710015661083
6,97.8723,375.0,23.42,62.84,0,15.5164,8.10291,13.663,2.4881169993022922
7,96.9072,368.8,23.42,63.14,0,15.7471,8.04993,13.1846,2.1400529985839967
8,94.9495,375.0,23.42,63.48,0,15.7457,8.65467,12.5624,3.425238999625435
9,97,381.2,23.42,63.47,0,15.9715,8.23753,12.9971,3.056495999771869
10,96.875,368.8,23.41,63.47,0,16.1014,8.21751,12.7778,3.7401400004455354
11,95.8763,400.0,2